<div dir="rtl" style="font-family: 'IRANSans', 'Tahoma', sans-serif;">


# حل سیستم معادلات با الگوریتم ژنتیک

این نوت‌بوک شامل یک حل‌کننده پیشرفته برای سیستم‌های معادلات خطی و غیرخطی با استفاده از الگوریتم ژنتیک است.

# راهنمای کامل حل‌کننده ژنتیک سیستم معادلات

## فهرست مطالب
1. [كلاس حل معادلات](#EnhancedGeneticEquationSolver)
2. [تحلیل عمیق تابع شایستگی](#Fitness-Function)
3. [روش‌های جایگزین تابع شایستگی](#alternative-methods-of-the-fitness-function)
4. [عوامل مؤثر در الگوریتم ژنتیک](#importance-of-factors-affecting-genetic-algorithms)

---

 نویسنده: [محمدمهدی شریف‌بیگی][(sharifbeigymohammad@gmail.com)](https://github.com/MohammadMahdi-Sharifbeigy)
 
</div>

In [ ]:
import numpy as np
import random
from fractions import Fraction
import re
import time

<div dir="rtl" style="font-family: 'IRANSans', 'Tahoma', sans-serif;">

### EnhancedGeneticEquationSolver

این کلاس قلب اصلی برنامه است و شامل اجزای زیر می‌باشد:

#### 1. سازنده کلاس (`__init__`)

**پارامترهای ورودی:**
- `pop_size=500`: اندازه جمعیت - تعداد کاندیداهای هر نسل
- `generations=5000`: حداکثر تعداد نسل‌ها
- `mutation_rate=0.2`: نرخ جهش (20%)
- `elite_size=50`: تعداد بهترین‌هایی که به نسل بعد منتقل می‌شوند
- `var_range=(-100, 100)`: محدوده مقادیر متغیرها
- `tournament_size=5`: اندازه تورنمنت برای انتخاب
- `stagnation_limit=200`: حد رکود قبل از راه‌اندازی مجدد
- `adaptive_mutation=True`: استفاده از نرخ جهش تطبیقی
- `fraction_precision=True`: تمایل به کسرهای معمول

#### 2. تابع شایستگی (`fitness_function`)

**هدف:** ارزیابی کیفیت هر کروموزوم (مجموعه مقادیر متغیرها)

<div dir='ltr'> 

```python
def fitness_function(self, chromosome, equations):
    total_error = 0
    for eq in equations:
        try:
            result = eq(*chromosome)
            total_error += result**2  # خطای مربعی
        except:
            total_error += 1e6  # جریمه سنگین
    
    if total_error < 1e-15:
        return float('inf')
    
    return 1.0 / total_error
```

</div>

#### 3. ایجاد جمعیت اولیه (`initialize_population`)

**استراتژی متنوع:**
- 25% اعداد صحیح
- 25% اعداد کوچک
- 25% کسرهای معمول
- 25% اعداد اعشاری متنوع

#### 4. ارزیابی جمعیت (`evaluate_population`)

همه افراد جمعیت را ارزیابی کرده و بر اساس شایستگی مرتب می‌کند.

#### 5. انتخاب والدین (`selection`)

از روش تورنمنت استفاده می‌کند:
- 90% احتمال انتخاب بهترین
- 10% احتمال انتخاب دومین بهترین (تنوع)

#### 6. ترکیب (`crossover`)

سه روش مختلف:
- **یکنواخت**: هر ژن از یکی از والدین
- **حسابی**: میانگین وزنی والدین
- **نقطه‌ای**: ترکیب در نقطه خاص

#### 7. جهش (`mutation`)

پنج استراتژی مختلف:
- **تصادفی**: مقدار کاملاً جدید
- **تغییر کوچک**: تنظیم ظریف
- **کسر**: تبدیل به کسر معمول
- **صفر**: نزدیک به صفر
- **تغییر علامت**: منفی کردن

#### 8. تولید نسل بعدی (`next_generation`)

ترکیب نخبگان و فرزندان جدید.

#### 9. راه‌اندازی مجدد (`restart_population`)

هنگام رکود، جمعیت جدید با تنوع بیشتر ایجاد می‌کند.

#### 10. تبدیل به کسر (`approximate_fractions`)

اعداد اعشاری را به کسرهای دقیق تبدیل می‌کند.

#### 11. حل اصلی (`solve`)

مدیریت کل فرآیند حل با قابلیت‌های:
- نظارت بر پیشرفت
- تشخیص رکود
- راه‌اندازی مجدد خودکار
- گزارش‌دهی مفصل


</div>

In [ ]:
class EnhancedGeneticEquationSolver:
    def __init__(self, pop_size=500, generations=5000, mutation_rate=0.2, elite_size=50,
                 var_range=(-100, 100), tournament_size=5, stagnation_limit=200, 
                 adaptive_mutation=True, fraction_precision=True):
        """
        Enhanced genetic algorithm for solving equation systems
        
        pop_size: Population size (increased)
        generations: Maximum number of generations
        mutation_rate: Base mutation rate
        elite_size: Number of elite solutions to preserve
        var_range: Range of variable values
        tournament_size: Tournament size for selection
        stagnation_limit: Max generations without improvement before restart
        adaptive_mutation: Whether to use adaptive mutation rates
        fraction_precision: Whether to bias toward common fractions
        """
        self.pop_size = pop_size
        self.generations = generations
        self.base_mutation_rate = mutation_rate
        self.current_mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.var_range = var_range
        self.tournament_size = tournament_size
        self.stagnation_limit = stagnation_limit
        self.adaptive_mutation = adaptive_mutation
        self.fraction_precision = fraction_precision
        # Common fractions to bias toward (to improve exact fraction finding)
        self.common_fractions = [
            Fraction(1, i) for i in range(1, 21)  # 1/1, 1/2, 1/3, ... 1/20
        ] + [Fraction(i, j) for i in range(-20, 21) for j in range(1, 21)
             if i != 0 and abs(i) <= abs(j)]  # Common fractions like 1/2, 2/3, 3/4, etc.
        
    def fitness_function(self, chromosome, equations):
        """
        Calculate fitness based on how well the chromosome solves all equations
        
        chromosome: Chromosome (variable values)
        equations: Equation functions
        :return: Fitness score (higher is better)
        """
        total_error = 0
        
        for eq in equations:
            try:
                result = eq(*chromosome)
                total_error += result**2  # Use squared error for better convergence
            except (ZeroDivisionError, ValueError, OverflowError, TypeError):
                # Heavily penalize invalid solutions
                total_error += 1e6
        
        # Avoid division by zero
        if total_error < 1e-15:
            return float('inf')
        
        return 1.0 / total_error
    
    def initialize_population(self, var_count):
        """
        Create diverse initial population with different strategies
        
        var_count: Number of variables
        :return: Initial population
        """
        population = []
        
        # Add solutions with integer values
        for _ in range(self.pop_size // 4):
            chromosome = [random.randint(self.var_range[0], self.var_range[1]) for _ in range(var_count)]
            # Avoid zeros if possible
            chromosome = [1 if gene == 0 else gene for gene in chromosome]
            population.append(chromosome)
            
        # Add solutions with small integer values (higher probability of being correct)
        for _ in range(self.pop_size // 4):
            chromosome = [random.randint(-10, 10) for _ in range(var_count)]
            chromosome = [1 if gene == 0 else gene for gene in chromosome]
            population.append(chromosome)
        
        # Add solutions with common fractions
        for _ in range(self.pop_size // 4):
            chromosome = []
            for _ in range(var_count):
                if random.random() < 0.7:  # 70% chance of common fraction
                    gene = float(random.choice(self.common_fractions))
                else:
                    gene = random.uniform(self.var_range[0], self.var_range[1])
                chromosome.append(gene)
            population.append(chromosome)
            
        # Add solutions with small decimals
        for _ in range(self.pop_size - len(population)):
            chromosome = [random.uniform(-10, 10) for _ in range(var_count)]
            chromosome = [0.1 if abs(gene) < 1e-10 else gene for gene in chromosome]
            population.append(chromosome)
            
        return population
    
    def evaluate_population(self, population, equations):
        """        
        population: Population of chromosomes
        equations: Equation functions
        :return: Ranked population with fitness scores
        """
        fitness_scores = []
        for chromosome in population:
            fitness = self.fitness_function(chromosome, equations)
            fitness_scores.append((chromosome, fitness))
        
        return sorted(fitness_scores, key=lambda x: x[1], reverse=True)
    
    def selection(self, ranked_population):
        """        
        ranked_population: Ranked population
        :return: Selected chromosome
        """
        tournament = random.sample(ranked_population, self.tournament_size)
        # Select the best with high probability, but sometimes take second best
        if random.random() < 0.9:
            return max(tournament, key=lambda x: x[1])[0]
        else:
            # Sort tournament and take second best if available
            sorted_tournament = sorted(tournament, key=lambda x: x[1], reverse=True)
            if len(sorted_tournament) > 1:
                return sorted_tournament[1][0]
            return sorted_tournament[0][0]
    
    def crossover(self, parent1, parent2):
        """        
        parent1: First parent
        parent2: Second parent
        :return: Child chromosome
        """
        # Choose crossover strategy randomly
        strategy = random.choice(['uniform', 'arithmetic', 'single_point'])
        
        if strategy == 'uniform':
            # Uniform crossover (each gene from either parent with 50% chance)
            child = [p1 if random.random() < 0.5 else p2 for p1, p2 in zip(parent1, parent2)]
            
        elif strategy == 'arithmetic':
            # Arithmetic crossover (weighted average of parents)
            alpha = random.random()  # Random weight
            child = [alpha * p1 + (1 - alpha) * p2 for p1, p2 in zip(parent1, parent2)]
            
        else:  # single_point
            point = random.randint(1, len(parent1) - 1)
            child = parent1[:point] + parent2[point:]
            
        return child
    
    def mutation(self, chromosome, generation, max_generations):
        """        
        chromosome: Chromosome to mutate
        generation: Current generation number
        max_generations: Maximum generations
        :return: Mutated chromosome
        """
        mutated = chromosome.copy()
        
        # Adjust mutation rate if adaptive mutation is enabled
        if self.adaptive_mutation:
            # Start with higher mutation rate, then decrease
            progress = min(1.0, generation / (max_generations * 0.7))
            self.current_mutation_rate = self.base_mutation_rate * (1.0 - 0.6 * progress)
            
            # Increase if population seems stuck (handled in solve method)
        
        for i in range(len(mutated)):
            # Each gene mutates with current rate
            if random.random() < self.current_mutation_rate:
                strategy = random.choices(
                    ['random', 'small_change', 'fraction', 'zero', 'sign_flip'],
                    weights=[0.3, 0.4, 0.2, 0.05, 0.05],
                    k=1
                )[0]
                
                if strategy == 'random':
                    # Completely new random value
                    mutated[i] = random.uniform(self.var_range[0], self.var_range[1])
                    
                elif strategy == 'small_change':
                    # Small adjustment to current value
                    magnitude = abs(mutated[i]) * 0.1 if mutated[i] != 0 else 0.1
                    mutated[i] += random.uniform(-magnitude, magnitude)
                    
                elif strategy == 'fraction' and self.fraction_precision:
                    # Set to a common fraction
                    mutated[i] = float(random.choice(self.common_fractions))
                    
                elif strategy == 'zero':
                    # Set close to zero (not exactly to avoid division errors)
                    mutated[i] = random.choice([0.1, -0.1])
                    
                elif strategy == 'sign_flip':
                    # Flip sign
                    mutated[i] = -mutated[i]
        
        return mutated
    
    def next_generation(self, current_ranked_population, generation, max_generations):
        """        
        current_ranked_population: Current ranked population
        generation: Current generation number
        max_generations: Maximum generations
        :return: New population
        """
        # Get elite solutions
        elites = [x[0] for x in current_ranked_population[:self.elite_size]]
        
        # Create children using selection, crossover and mutation
        children = []
        while len(children) < self.pop_size - len(elites):
            # Select parents
            parent1 = self.selection(current_ranked_population)
            parent2 = self.selection(current_ranked_population)
            
            # Create child via crossover
            child = self.crossover(parent1, parent2)
            
            # Mutate child
            child = self.mutation(child, generation, max_generations)
            
            children.append(child)
        
        # New population with elites and children
        new_population = elites + children
        return new_population
    
    def restart_population(self, best_solution, var_count):
        """        
        best_solution: Best solution found so far
        var_count: Number of variables
        :return: New diverse population
        """
        # Initialize new population
        new_population = self.initialize_population(var_count)
        
        # Replace some individuals with variations of best solution
        num_seeds = min(self.pop_size // 10, 50)  # 10% or up to 50 individuals
        
        for i in range(num_seeds):
            # Create variation of best solution
            variation = best_solution.copy()
            
            # Apply small mutations to create diversity
            for j in range(var_count):
                if random.random() < 0.3:  # 30% chance to mutate each gene
                    magnitude = abs(variation[j]) * 0.1 if variation[j] != 0 else 0.1
                    variation[j] += random.uniform(-magnitude, magnitude)
            
            # Replace random individual in population
            new_population[random.randint(0, self.pop_size - 1)] = variation
        
        return new_population
    
    def approximate_fractions(self, solution, max_denominator=100):
        """        
        solution: Solution as floats
        max_denominator: Maximum denominator
        :return: Solution as fractions
        """
        result = []
        for value in solution:
            # Try to find closest common fraction first
            if self.fraction_precision:
                best_fraction = None
                min_diff = float('inf')
                
                for frac in self.common_fractions:
                    diff = abs(float(frac) - value)
                    if diff < min_diff and diff < 1e-2:  # Within reasonable tolerance
                        min_diff = diff
                        best_fraction = frac
                
                if best_fraction is not None:
                    result.append(best_fraction)
                    continue
            
            # Fall back to standard fraction approximation
            fraction = Fraction(value).limit_denominator(max_denominator)
            result.append(fraction)
        
        return result
    
    def solve(self, equations, var_count):
        """        
        equations: Equation functions
        var_count: Number of variables
        :return: Best solution as fractions
        """
        start_time = time.time()
        
        # Initialize
        population = self.initialize_population(var_count)
        best_solution = None
        best_fitness = -float('inf')
        generations_without_improvement = 0
        restart_count = 0
        
        for generation in range(self.generations):
            # Evaluate population
            ranked_population = self.evaluate_population(population, equations)
            
            # Check best solution
            current_best = ranked_population[0]
            current_fitness = current_best[1]
            
            if current_fitness > best_fitness:
                best_solution = current_best[0]
                best_fitness = current_fitness
                generations_without_improvement = 0
                
                # If exact solution found, break early
                if best_fitness > 1e10:
                    print(f"Exact solution found in generation {generation}.")
                    break
            else:
                generations_without_improvement += 1
            
            # Report progress
            if generation % 100 == 0 or generation == self.generations - 1:
                error = 1.0 / best_fitness if best_fitness != float('inf') else 0
                elapsed_time = time.time() - start_time
                print(f"Generation {generation}: Best error = {error:.10f}, Time: {elapsed_time:.2f}s")
            
            # Check if population is stuck
            if generations_without_improvement >= self.stagnation_limit:
                if restart_count < 5:  # Limit number of restarts
                    # Restart population but keep best solution
                    print(f"Population stuck for {generations_without_improvement} generations. Restarting...")
                    population = self.restart_population(best_solution, var_count)
                    generations_without_improvement = 0
                    restart_count += 1
                    
                    # Increase mutation rate temporarily
                    if self.adaptive_mutation:
                        self.current_mutation_rate = min(0.5, self.base_mutation_rate * 2)
                else:
                    # If we've restarted too many times without success, try to refine best solution
                    print("Multiple restarts without significant improvement. Focusing on refinement...")
                    # Focus on small changes to best solution
                    population = []
                    for _ in range(self.pop_size):
                        variation = best_solution.copy()
                        for i in range(len(variation)):
                            # Small random adjustments
                            if random.random() < 0.3:
                                magnitude = abs(variation[i]) * 0.01 if variation[i] != 0 else 0.01
                                variation[i] += random.uniform(-magnitude, magnitude)
                        population.append(variation)
                    generations_without_improvement = 0
            
            # Create next generation
            population = self.next_generation(ranked_population, generation, self.generations)
        
        # Convert solution to fractions
        solution_fractions = self.approximate_fractions(best_solution)
        
        error = 1.0 / best_fitness if best_fitness != float('inf') else 0
        elapsed_time = time.time() - start_time
        print(f"Best solution: {solution_fractions}")
        print(f"Final error: {error:.10f}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        
        return solution_fractions

<div dir="rtl" style="font-family: 'IRANSans', 'Tahoma', sans-serif;">

## Fitness Function

### چرا از این تابع شایستگی استفاده شد؟

<div dir="ltr">

```python
def fitness_function(self, chromosome, equations):
    total_error = 0
    for eq in equations:
        try:
            result = eq(*chromosome)
            total_error += result**2  # خطای مربعی
        except:
            total_error += 1e6
    
    if total_error < 1e-15:
        return float('inf')
    
    return 1.0 / total_error
```
</div>

### دلایل انتخاب این روش:

#### 1. **استفاده از خطای مربعی (`result**2`)**
- **دلیل**: خطاهای بزرگ بیشتر تنبیه می‌شوند
- **مزیت**: همگرایی سریع‌تر به جواب دقیق
- **مثال**: اگر دو کروموزوم خطاهای 0.1 و 0.01 داشته باشند:
  - روش خطی: 0.1 vs 0.01 (تفاوت 10 برابری)
  - روش مربعی: 0.01 vs 0.0001 (تفاوت 100 برابری)

#### 2. **معکوس خطا (`1.0 / total_error`)**
- **دلیل**: تبدیل مسئله minimization به maximization
- **مزیت**: الگوریتم ژنتیک معمولاً برای بیشینه‌سازی طراحی شده
- **نتیجه**: خطای کمتر = شایستگی بیشتر

#### 3. **مدیریت استثناء**
- **جریمه سنگین** (`1e6`) برای کروموزوم‌های نامعتبر
- **جلوگیری** از تقسیم بر صفر، سرریز عددی، و خطاهای محاسباتی

#### 4. **تشخیص جواب دقیق**
- اگر خطا کمتر از `1e-15` باشد، شایستگی `inf` می‌شود
- **نتیجه**: توقف زودهنگام در صورت یافتن جواب دقیق

---

## Alternative methods of the fitness function

روش‌های جایگزین تابع شایستگی


### 1. **روش خطای خطی (Linear Error)**

<div dir="ltr">

```python
def linear_fitness(self, chromosome, equations):
    total_error = 0
    for eq in equations:
        total_error += abs(eq(*chromosome))
    return 1.0 / (1.0 + total_error)
```
</div>

**مزایا:**
- ساده‌تر در پیاده‌سازی
- حساسیت کمتر به نویز
- پایداری عددی بهتر

**معایب:**
- همگرایی کندتر
- تمایز کمتر بین جواب‌های خوب و عالی
- ممکن است در جواب‌های تقریبی گیر کند

### 2. **روش خطای لگاریتمی (Logarithmic Error)**

<div dir="ltr">

```python
def log_fitness(self, chromosome, equations):
    total_error = 0
    for eq in equations:
        error = abs(eq(*chromosome))
        if error > 0:
            total_error += math.log(1 + error)
    return 1.0 / (1.0 + total_error)
```

</div>

**مزایا:**
- کاهش اثر خطاهای بزرگ
- تعادل بهتر بین دقت و پایداری
- مناسب برای مسائل با مقیاس‌های مختلف

**معایب:**
- پیچیدگی محاسباتی بیشتر
- ممکن است دقت نهایی کمتری داشته باشد
- نیاز به تنظیم پارامترهای اضافی

### 3. **روش وزن‌دار (Weighted Error)**

<div dir="ltr">

```python
def weighted_fitness(self, chromosome, equations, weights=None):
    if weights is None:
        weights = [1.0] * len(equations)
    
    total_error = 0
    for eq, weight in zip(equations, weights):
        error = eq(*chromosome)
        total_error += weight * (error ** 2)
    
    return 1.0 / (1.0 + total_error)
```

</div>

**مزایا:**
- امکان اولویت‌بندی معادلات مختلف
- انعطاف‌پذیری بالا
- کنترل بهتر بر فرآیند همگرایی

**معایب:**
- نیاز به تعیین وزن‌های مناسب
- پیچیدگی بیشتر در تنظیم
- ممکن است به برخی معادلات بی‌توجهی کند

### 4. **روش نرمالایز شده (Normalized Error)**

<div dir="ltr">

```python
def normalized_fitness(self, chromosome, equations):
    errors = []
    for eq in equations:
        errors.append(abs(eq(*chromosome)))
    
    # نرمالایز بر اساس میانگین و انحراف معیار
    mean_error = sum(errors) / len(errors)
    if mean_error == 0:
        return float('inf')
    
    normalized_error = sum(e / mean_error for e in errors)
    return 1.0 / (1.0 + normalized_error)
```

</div>

**مزایا:**
- تعادل بهتر بین معادلات مختلف
- کاهش اثر معادلات با مقیاس بزرگ
- عملکرد یکنواخت‌تر

**معایب:**
- محاسبات اضافی برای نرمالایز
- ممکن است اطلاعاتی از دست برود
- پیچیدگی در پیاده‌سازی

### 5. **روش تطبیقی (Adaptive Fitness)**

<div dir="ltr">

```python
def adaptive_fitness(self, chromosome, equations, generation):
    total_error = 0
    
    # ضریب تطبیقی بر اساس نسل
    adaptive_power = 2.0 + (generation / 1000.0)
    
    for eq in equations:
        error = abs(eq(*chromosome))
        total_error += error ** adaptive_power
    
    return 1.0 / (1.0 + total_error)
```

</div>

**مزایا:**
- تطبیق خودکار با پیشرفت الگوریتم
- شروع با جستجوی گسترده، ادامه با جستجوی دقیق
- بهینه‌سازی مراحل مختلف حل

**معایب:**
- پیچیدگی زیاد
- نیاز به تنظیم پارامترهای بیشتر
- ممکن است پیش‌بینی‌ناپذیر باشد

### 6. **روش ترکیبی چندگانه (Multi-Objective)**

<div dir="ltr">

```python
def multi_objective_fitness(self, chromosome, equations):
    # هدف 1: کمینه‌سازی خطا
    total_error = sum(eq(*chromosome)**2 for eq in equations)
    
    # هدف 2: سادگی جواب (ترجیح اعداد کوچک)
    complexity = sum(abs(x) for x in chromosome)
    
    # هدف 3: تعداد کسرهای ساده
    fraction_score = sum(1 for x in chromosome 
                        if abs(x - round(x*4)/4) < 1e-3)
    
    # ترکیب وزن‌دار
    fitness = (1.0 / (1 + total_error)) * 0.7 + \
              (1.0 / (1 + complexity)) * 0.2 + \
              fraction_score * 0.1
    
    return fitness
```

</div>

**مزایا:**
- در نظر گیری چندین معیار همزمان
- تولید جواب‌های عملی‌تر
- انعطاف‌پذیری بسیار بالا

**معایب:**
- پیچیدگی زیاد در طراحی و تنظیم
- ممکن است همگرایی آهسته‌تر داشته باشد
- نیاز به دانش تخصصی برای تنظیم وزن‌ها

---

## جمع‌بندی و توصیه‌ها

### چرا روش فعلی (خطای مربعی معکوس) بهترین انتخاب است؟

1. **سادگی و کارایی**: پیاده‌سازی ساده با عملکرد عالی
2. **همگرایی سریع**: خطاهای بزرگ شدیداً تنبیه می‌شوند
3. **دقت بالا**: قادر به یافتن جواب‌های بسیار دقیق
4. **پایداری**: مدیریت مناسب حالات استثنایی

### کی از روش‌های دیگر استفاده کنیم؟

- **خطای خطی**: برای مسائل حساس به نویز
- **وزن‌دار**: وقتی برخی معادلات اهمیت بیشتری دارند
- **نرمالایز**: برای معادلات با مقیاس‌های مختلف
- **تطبیقی**: برای مسائل خیلی پیچیده
- **چندهدفه**: وقتی کیفیت جواب اهمیت دارد

این انتخاب بستگی به نوع مسئله، دقت مورد نیاز، و منابع محاسباتی در دسترس دارد.

</div>

<div dir="rtl" style="font-family: 'IRANSans', 'Tahoma', sans-serif;">


### سلول سوم: توابع کمکی

#### 1. تجزیه معادلات (`parse_advanced_equation`)

معادلات متنی را به توابع پایتون تبدیل می‌کند.

#### 2. نمایش کسرها (`print_fraction`)

کسرها را به شکل خوانا نمایش می‌دهد.

#### 3. حل سیستم (`solve_equations`)

کل فرآیند حل را مدیریت می‌کند.

#### 4. تست جواب (`test_known_solution`)

صحت جواب‌های شناخته شده را بررسی می‌کند.

#### 5. توابع تست

سه سیستم نمونه با پیچیدگی متفاوت:
- دو متغیر
- سه متغیر  
- چهار متغیر

---

</div>

In [ ]:
def parse_advanced_equation(equation_str, var_names):
    """
    Convert an equation string to a function
    
    equation_str: Equation string
    var_names: Variable names
    return: Function that returns the error
    """
    # Split left and right sides
    sides = equation_str.replace(" ", "").split("=")
    left_side = sides[0]
    right_side = sides[1]
    
    # Create error function
    def equation_error(*vars):
        # Create variable dictionary
        var_dict = {name: value for name, value in zip(var_names, vars)}
        
        try:
            # Calculate both sides of the equation
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            # Return absolute error
            return abs(left_val - right_val)
        except (ZeroDivisionError, ValueError, OverflowError):
            # Return large error
            return 1e6
    
    return equation_error


def parse_equations_system(equations_str, var_names):
    """
    Convert equation strings to functions
    
    equations_str: List of equation strings
    var_names: Variable names
    :return: List of error functions
    """
    equation_functions = []
    for eq_str in equations_str:
        equation_functions.append(parse_advanced_equation(eq_str, var_names))
    return equation_functions


def print_fraction(frac):
    """Display fraction in readable format"""
    if frac.denominator == 1:
        return str(frac.numerator)
    else:
        return f"{frac.numerator}/{frac.denominator}"


def solve_equations(equations, var_names):
    """Solve equation system"""
    # Convert equations to functions
    equations_funcs = parse_equations_system(equations, var_names)
    
    solver = EnhancedGeneticEquationSolver(
        pop_size=500,             
        generations=1000,         
        mutation_rate=0.2,      
        elite_size=50,          
        var_range=(-100, 100),    
        tournament_size=5,      
        stagnation_limit=200,     
        adaptive_mutation=True,   
        fraction_precision=True   
    )
    
    solution = solver.solve(equations_funcs, len(var_names))
    
    print("\nFinal solution:")
    for var, val in zip(var_names, solution):
        print(f"{var} = {print_fraction(val)}")
    
    print("\nVerifying solution in equations:")
    for i, eq_str in enumerate(equations):
        try:
            # Create variable dictionary
            var_dict = {name: float(value) for name, value in zip(var_names, solution)}
            
            # Calculate both sides
            sides = eq_str.replace(" ", "").split("=")
            left_side = sides[0]
            right_side = sides[1]
            
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            print(f"Equation {i+1}: {left_val} ≈ {right_val}, error: {abs(left_val - right_val)}")
        except Exception as e:
            print(f"Equation {i+1}: Error in verification - {str(e)}")
    
    return solution


def test_known_solution(equations, var_names, known_solution):
    """Test the known solution in the equations"""
    print("\n=== Validating Known Solution ===")
    
    # Display known solution
    print("Known solution:")
    for var, val in zip(var_names, known_solution):
        print(f"{var} = {print_fraction(val)}")
    
    # Verify in equations
    print("\nVerifying in equations:")
    for i, eq_str in enumerate(equations):
        try:
            # Create variable dictionary
            var_dict = {name: float(value) for name, value in zip(var_names, known_solution)}
            
            # Calculate both sides
            sides = eq_str.replace(" ", "").split("=")
            left_side = sides[0]
            right_side = sides[1]
            
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            print(f"Equation {i+1}: {left_val} ≈ {right_val}, error: {abs(left_val - right_val)}")
        except Exception as e:
            print(f"Equation {i+1}: Error in verification - {str(e)}")


def solve_2var_system():
    """Solve the 2-variable equation system from the image"""
    print("\n=== Solving 2-variable equation system ===")
    
    equations = [
        "x + 2*y = 4",
        "4*y + 4*x= 12",
    ]
    
    var_names = ['x', 'y']
    
    # Known solution from image
    known_solution = [
        Fraction(2, 1),
        Fraction(1, 1)
    ]
    
    # Verify known solution
    test_known_solution(equations, var_names, known_solution)
    
    # Solve with enhanced genetic algorithm
    solve_equations(equations, var_names)

def solve_3var_system():
    """Solve the 3-variable equation system from the image"""
    print("\n=== Solving 3-variable equation system ===")
    
    equations = [
        "6*x - 2*y + 8*z = 20",
        "y + 8*x * z = -1",
        "2*z * (6/x) + (3/2)*y = 6"
    ]
    
    var_names = ['x', 'y', 'z']
    
    # Known solution from image
    known_solution = [
        Fraction(2, 3),   # x = 2/3
        Fraction(-5, 1),  # y = -5
        Fraction(3, 4)    # z = 3/4
    ]
    
    # Verify known solution
    test_known_solution(equations, var_names, known_solution)
    
    # Solve with enhanced genetic algorithm
    solve_equations(equations, var_names)

def solve_4var_system():
    """Solve the 4-variable system"""
    print("\n=== Solving 4-variable equation system ===")
    
    equations = [
        "(1/15)*x - 2*y - 15*z - (4/5)*t = 3",
        "-(5/2)*x - (9/4)*y + 12*z - t = 17",
        "-13*x + (3/10)*y - 6*z - (2/5)*t = 17",
        "(1/2)*x + 2*y + (7/4)*z + (4/3)*t = -9"
    ]
    
    var_names = ['x', 'y', 'z', 't']
    
    # Known solution from image
    known_solution = [
        Fraction(-3, 2),  # x = -3/2
        Fraction(-7, 2),  # y = -7/2
        Fraction(1, 3),   # z = 1/3
        Fraction(-11, 8)  # t = -11/8
    ]
    
    test_known_solution(equations, var_names, known_solution)
    
    solve_equations(equations, var_names)


# solve_2var_system()
# solve_3var_system()
solve_4var_system()


=== Solving 4-variable equation system ===

=== Validating Known Solution ===
Known solution:
x = -3/2
y = -7/2
z = 1/3
t = -11/8

Verifying in equations:
Equation 1: 3.0000000000000004 ≈ 3, error: 4.440892098500626e-16
Equation 2: 17.0 ≈ 17, error: 0.0
Equation 3: 17.0 ≈ 17, error: 0.0
Equation 4: -9.0 ≈ -9, error: 0.0
Generation 0: Best error = 259.1041939076, Time: 0.05s
Generation 100: Best error = 0.0332690146, Time: 5.32s
Generation 200: Best error = 0.0034556077, Time: 10.56s
Generation 300: Best error = 0.0018342813, Time: 15.85s
Generation 400: Best error = 0.0012318086, Time: 20.97s
Generation 500: Best error = 0.0008376778, Time: 26.05s
Generation 600: Best error = 0.0006275433, Time: 31.12s
Generation 700: Best error = 0.0004000948, Time: 36.22s
Generation 800: Best error = 0.0003237690, Time: 41.40s
Generation 900: Best error = 0.0002212688, Time: 46.81s
Generation 999: Best error = 0.0001713835, Time: 52.38s
Best solution: [Fraction(-3, 2), Fraction(-251, 72), Fraction(1

<div dir="rtl" style="font-family: 'IRANSans', 'Tahoma', sans-serif;">

# importance of Factors affecting genetic algorithms

## فهرست عوامل به ترتیب اولویت

1. [روش انتخاب (Selection Strategy)](#1-روش-انتخاب-selection-strategy)
2. [عملگر ترکیب (Crossover Operator)](#2-عملگر-ترکیب-crossover-operator)
3. [استراتژی جهش (Mutation Strategy)](#3-استراتژی-جهش-mutation-strategy)
4. [مدیریت جمعیت (Population Management)](#4-مدیریت-جمعیت-population-management)
5. [پارامترهای کنترلی (Control Parameters)](#5-پارامترهای-کنترلی-control-parameters)
6. [نحوه ایجاد جمعیت اولیه (Initialization Strategy)](#6-نحوه-ایجاد-جمعیت-اولیه-initialization-strategy)
7. [معیارهای توقف (Termination Criteria)](#7-معیارهای-توقف-termination-criteria)

---

## 1. روش انتخاب (Selection Strategy)

### تأثیر: **بالا** (اولویت 1)
**دلیل**: تعیین‌کننده نحوه انتشار ژن‌های خوب در جمعیت

### روش استفاده شده در کد:

<div dir='ltr'>

```python
def selection(self, ranked_population):
    tournament = random.sample(ranked_population, self.tournament_size)
    if random.random() < 0.9:
        return max(tournament, key=lambda x: x[1])[0]
    else:
        sorted_tournament = sorted(tournament, key=lambda x: x[1], reverse=True)
        return sorted_tournament[1][0] if len(sorted_tournament) > 1 else sorted_tournament[0][0]
```

</div>

**نوع**: Tournament Selection با اندازه 5

### روش‌های جایگزین:

#### 1.1. Roulette Wheel Selection

<div dir="ltr">

```python
def roulette_wheel_selection(self, ranked_population):
    total_fitness = sum(fitness for _, fitness in ranked_population)
    pick = random.uniform(0, total_fitness)
    current = 0
    for chromosome, fitness in ranked_population:
        current += fitness
        if current > pick:
            return chromosome
```

</div>

**مزایا:**
- احتمال انتخاب متناسب با شایستگی
- حفظ تنوع بهتر
- پیاده‌سازی ساده

**معایب:**
- حساس به scaling مقادیر شایستگی
- ممکن است افراد ضعیف زیاد انتخاب شوند
- مشکل با شایستگی‌های منفی

#### 1.2. Rank-Based Selection

<div dir="ltr">

```python
def rank_selection(self, ranked_population):
    n = len(ranked_population)
    # اختصاص rank به جای استفاده از مقدار دقیق fitness
    ranks = list(range(n, 0, -1))  # بهترین = n، بدترین = 1
    total_rank = sum(ranks)
    pick = random.uniform(0, total_rank)
    current = 0
    for i, (chromosome, _) in enumerate(ranked_population):
        current += ranks[i]
        if current > pick:
            return chromosome
```

</div>

**مزایا:**
- مستقل از مقیاس شایستگی
- کنترل بهتر فشار انتخاب
- جلوگیری از dominance زودرس

**معایب:**
- از دست دادن اطلاعات دقیق شایستگی
- محاسبات بیشتر برای رتبه‌بندی
- ممکن است همگرایی کندتر باشد

#### 1.3. Stochastic Universal Sampling (SUS)

<div dir="ltr">

```python
def sus_selection(self, ranked_population, num_parents):
    total_fitness = sum(fitness for _, fitness in ranked_population)
    distance = total_fitness / num_parents
    start = random.uniform(0, distance)
    points = [start + i * distance for i in range(num_parents)]
    
    selected = []
    current = 0
    i = 0
    for point in points:
        while current < point and i < len(ranked_population):
            current += ranked_population[i][1]
            i += 1
        selected.append(ranked_population[i-1][0])
    return selected
```

</div>

**مزایا:**
- حداقل variance در انتخاب
- سرعت بالا برای انتخاب چندگانه
- توزیع عادلانه‌تر

**معایب:**
- پیچیدگی پیاده‌سازی
- کمتر شناخته شده
- نیاز به انتخاب تعداد والدین از قبل

### مقایسه با روش کد:

**Tournament Selection (کد فعلی):**
- ✅ کنترل آسان فشار انتخاب
- ✅ عملکرد سریع
- ✅ مستقل از scaling
- ❌ تنوع کمتر نسبت به Roulette
- ❌ ممکن است در optimum محلی گیر کند

---

## 2. عملگر ترکیب (Crossover Operator)

### تأثیر: **بالا** (اولویت 2)
**دلیل**: تعیین نحوه ترکیب اطلاعات والدین

### روش استفاده شده در کد:

<div dir="ltr">

```python
def crossover(self, parent1, parent2):
    strategy = random.choice(['uniform', 'arithmetic', 'single_point'])
    
    if strategy == 'uniform':
        child = [p1 if random.random() < 0.5 else p2 for p1, p2 in zip(parent1, parent2)]
    elif strategy == 'arithmetic':
        alpha = random.random()
        child = [alpha * p1 + (1 - alpha) * p2 for p1, p2 in zip(parent1, parent2)]
    else:  # single_point
        point = random.randint(1, len(parent1) - 1)
        child = parent1[:point] + parent2[point:]
    
    return child
```

</div>

### روش‌های جایگزین:

#### 2.1. Two-Point Crossover

<div dir="ltr">

```python
def two_point_crossover(self, parent1, parent2):
    length = len(parent1)
    point1 = random.randint(1, length - 2)
    point2 = random.randint(point1 + 1, length - 1)
    
    child = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    return child
```

</div>

**مزایا:**
- تنوع بیشتر نسبت به single-point
- حفظ بهتر ساختار ژن‌ها
- کنترل بهتر disruption

**معایب:**
- پیچیدگی بیشتر
- ممکن است ساختارهای خوب را بشکند
- نیاز به تنظیم دقیق

#### 2.2. Blend Crossover (BLX-α)

<div dir="ltr">

```python
def blx_crossover(self, parent1, parent2, alpha=0.5):
    child = []
    for p1, p2 in zip(parent1, parent2):
        min_val = min(p1, p2)
        max_val = max(p1, p2)
        range_val = max_val - min_val
        
        lower_bound = min_val - alpha * range_val
        upper_bound = max_val + alpha * range_val
        
        child.append(random.uniform(lower_bound, upper_bound))
    return child
```

</div>

**مزایا:**
- مناسب برای متغیرهای پیوسته
- اکتشاف فضای جستجو بهتر
- تولید تنوع بیشتر

**معایب:**
- ممکن است از محدوده خارج شود
- نیاز به تنظیم پارامتر α
- ممکن است همگرایی کندتر باشد

#### 2.3. Simulated Binary Crossover (SBX)

<div dir="ltr">

```python
def sbx_crossover(self, parent1, parent2, eta=2.0):
    child1, child2 = [], []
    for p1, p2 in zip(parent1, parent2):
        if random.random() <= 0.5:
            if abs(p1 - p2) > 1e-14:
                if p1 < p2:
                    y1, y2 = p1, p2
                else:
                    y1, y2 = p2, p1
                
                rand = random.random()
                beta = 1.0 + (2.0 / (y2 - y1))
                
                if rand <= (1.0 / beta):
                    betaq = (rand * beta) ** (1.0 / (eta + 1.0))
                else:
                    betaq = (1.0 / (2.0 - rand * beta)) ** (1.0 / (eta + 1.0))
                
                c1 = 0.5 * ((y1 + y2) - betaq * (y2 - y1))
                c2 = 0.5 * ((y1 + y2) + betaq * (y2 - y1))
            else:
                c1, c2 = p1, p2
        else:
            c1, c2 = p1, p2
        
        child1.append(c1)
        child2.append(c2)
    
    return random.choice([child1, child2])
```

</div>

**مزایا:**
- شبیه‌سازی crossover باینری برای اعداد حقیقی
- کنترل دقیق spread فرزندان
- حفظ تنوع ژنتیک

**معایب:**
- پیچیدگی محاسباتی بالا
- نیاز به تنظیم پارامتر eta
- درک سخت‌تر

### مقایسه با روش کد:

**روش ترکیبی کد (Uniform + Arithmetic + Single-point):**
- ✅ تنوع بالا با سه استراتژی
- ✅ سادگی پیاده‌سازی
- ✅ تطبیق با انواع مسائل
- ❌ عدم کنترل دقیق بر هر روش
- ❌ ممکن است بهینه نباشد برای مسائل خاص

---

## 3. استراتژی جهش (Mutation Strategy)

### تأثیر: **متوسط تا بالا** (اولویت 3)
**دلیل**: حفظ تنوع و جلوگیری از optimum محلی

### روش استفاده شده در کد:

<div dir="ltr">

```python
def mutation(self, chromosome, generation, max_generations):
    strategy = random.choices(
        ['random', 'small_change', 'fraction', 'zero', 'sign_flip'],
        weights=[0.3, 0.4, 0.2, 0.05, 0.05],
        k=1
    )[0]
    
    # + نرخ جهش تطبیقی
    progress = min(1.0, generation / (max_generations * 0.7))
    self.current_mutation_rate = self.base_mutation_rate * (1.0 - 0.6 * progress)
```

</div>

### روش‌های جایگزین:

#### 3.1. Gaussian Mutation

<div dir="ltr">

```python
def gaussian_mutation(self, chromosome, generation, max_generations):
    mutated = chromosome.copy()
    
    # انحراف معیار کاهشی
    sigma = self.initial_sigma * (1.0 - generation / max_generations)
    
    for i in range(len(mutated)):
        if random.random() < self.current_mutation_rate:
            noise = random.gauss(0, sigma)
            mutated[i] += noise
            
            # اعمال محدودیت
            mutated[i] = max(self.var_range[0], min(self.var_range[1], mutated[i]))
    
    return mutated
```

</div>

**مزایا:**
- تغییرات طبیعی و هموار
- کنترل دقیق میزان تغییر
- مناسب برای fine-tuning

**معایب:**
- ممکن است از محدوده خارج شود
- نیاز به تنظیم σ
- تنوع کمتر در مراحل اولیه

#### 3.2. Polynomial Mutation

<div dir="ltr">

```python
def polynomial_mutation(self, chromosome, eta=20.0):
    mutated = chromosome.copy()
    
    for i in range(len(mutated)):
        if random.random() < self.current_mutation_rate:
            rand = random.random()
            xl, xu = self.var_range[0], self.var_range[1]
            
            if rand < 0.5:
                delta = (2.0 * rand) ** (1.0 / (eta + 1.0)) - 1.0
            else:
                delta = 1.0 - (2.0 * (1.0 - rand)) ** (1.0 / (eta + 1.0))
            
            mutated[i] += delta * (xu - xl)
            mutated[i] = max(xl, min(xu, mutated[i]))
    
    return mutated
```

</div>

**مزایا:**
- توزیع controlled تغییرات
- حفظ محدوده متغیرها
- عملکرد خوب در optimizationهای پیوسته

**معایب:**
- پیچیدگی ریاضی
- نیاز به تنظیم eta
- ممکن است برای مسائل گسسته مناسب نباشد

#### 3.3. Multi-Strategy Adaptive Mutation

<div dir="ltr">

```python
def adaptive_multi_mutation(self, chromosome, generation, max_generations, success_rates):
    strategies = ['gaussian', 'uniform', 'polynomial', 'cauchy']
    
    # انتخاب استراتژی بر اساس نرخ موفقیت
    weights = [success_rates.get(s, 0.25) for s in strategies]
    strategy = random.choices(strategies, weights=weights, k=1)[0]
    
    if strategy == 'gaussian':
        return self.gaussian_mutation(chromosome, generation, max_generations)
    elif strategy == 'uniform':
        return self.uniform_mutation(chromosome)
    # ... سایر استراتژی‌ها
```

</div>

**مزایا:**
- تطبیق خودکار با عملکرد
- ترکیب مزایای روش‌های مختلف
- یادگیری در طول اجرا

**معایب:**
- پیچیدگی زیاد
- نیاز به tracking موفقیت
- overhead محاسباتی

### مقایسه با روش کد:

**روش چندگانه کد (5 استراتژی + نرخ تطبیقی):**
- ✅ تنوع بسیار بالا
- ✅ تطبیق با مراحل مختلف
- ✅ مناسب برای مسائل مختلف
- ❌ عدم optimality برای مسائل خاص
- ❌ پیچیدگی در debugging

---

## 4. مدیریت جمعیت (Population Management)

### تأثیر: **متوسط** (اولویت 4)
**دلیل**: تعادل بین اکتشاف و بهره‌برداری

<div dir="ltr">

```python
def next_generation(self, current_ranked_population, generation, max_generations):
    # حفظ نخبگان
    elites = [x[0] for x in current_ranked_population[:self.elite_size]]
    
    # تولید فرزندان
    children = []
    while len(children) < self.pop_size - len(elites):
        parent1 = self.selection(current_ranked_population)
        parent2 = self.selection(current_ranked_population)
        child = self.crossover(parent1, parent2)
        child = self.mutation(child, generation, max_generations)
        children.append(child)
    
    return elites + children

# + restart mechanism
def restart_population(self, best_solution, var_count):
    # تولید جمعیت جدید + variations از بهترین
```

</div>

### روش‌های جایگزین:

#### 4.1. Steady-State GA

<div dir="ltr">

```python
def steady_state_replacement(self, population, new_individual):
    # جایگزینی یک فرد در هر بار
    evaluated_pop = self.evaluate_population(population + [new_individual], self.equations)
    
    # حذف بدترین
    evaluated_pop.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in evaluated_pop[:-1]]
```

</div>

**مزایا:**
- تغییر تدریجی جمعیت
- حفظ بهتر تنوع
- قابلیت parallel بهتر

**معایب:**
- همگرایی کندتر
- پیچیدگی در پیاده‌سازی
- کنترل سخت‌تر

#### 4.2. Island Model

<div dir="ltr">

```python
class IslandGA:
    def __init__(self, num_islands=4, migration_rate=0.1):
        self.islands = [GeneticSolver() for _ in range(num_islands)]
        self.migration_rate = migration_rate
    
    def migrate(self, generation):
        if generation % 50 == 0:  # هر 50 نسل
            for i in range(len(self.islands)):
                source = self.islands[i]
                target = self.islands[(i + 1) % len(self.islands)]
                
                # انتقال بهترین‌ها
                migrants = source.get_best(int(source.pop_size * self.migration_rate))
                target.replace_worst(migrants)
```

</div>

**مزایا:**
- تنوع بسیار بالا
- مقاومت در برابر optimum محلی
- قابلیت موازی‌سازی

**معایب:**
- پیچیدگی زیاد
- نیاز به منابع بیشتر
- تنظیم پارامترهای اضافی

#### 4.3. Age-Based Replacement

<div dir="ltr">

```python
def age_based_replacement(self, population, generation):
    # هر فرد سن دارد
    new_population = []
    for individual, age in population:
        if age < self.max_age:
            new_population.append((individual, age + 1))
        else:
            # جایگزینی فرد پیر با فرد جدید
            new_individual = self.create_random_individual()
            new_population.append((new_individual, 0))
    
    return new_population
```

</div>

**مزایا:**
- جلوگیری از stagnation
- تنوع مداوم
- ساده در پیاده‌سازی

**معایب:**
- ممکن است افراد خوب حذف شوند
- نیاز به تعیین حد سنی
- کنترل کمتر بر کیفیت

### مقایسه با روش کد:

**Elitism + Restart Mechanism:**
- ✅ حفظ بهترین جواب‌ها
- ✅ مقابله با stagnation
- ✅ سادگی پیاده‌سازی
- ❌ ممکن است تنوع کم شود
- ❌ restart ممکن است زودهنگام باشد

---

## 5. پارامترهای کنترلی (Control Parameters)

### تأثیر: **متوسط** (اولویت 5)
**دلیل**: تعیین رفتار کلی الگوریتم

### پارامترهای کد:

```python
pop_size=500           # اندازه جمعیت
generations=5000       # تعداد نسل‌ها  
mutation_rate=0.2      # نرخ جهش
elite_size=50          # تعداد نخبگان
tournament_size=5      # اندازه tournament
stagnation_limit=200   # حد رکود
```

### استراتژی‌های جایگزین:

#### 5.1. Parameter Control

<div dir="ltr">

```python
class AdaptiveParameters:
    def update_parameters(self, generation, fitness_variance):
        # تطبیق بر اساس تنوع fitness
        if fitness_variance < 0.01:  # جمعیت همگرا شده
            self.mutation_rate *= 1.5
            self.tournament_size = max(2, self.tournament_size - 1)
        else:
            self.mutation_rate *= 0.95
            self.tournament_size = min(10, self.tournament_size + 1)
```

</div>

#### 5.2. Multi-Population with Different Parameters

<div dir="ltr">

```python
class MultiPopulationGA:
    def __init__(self):
        self.populations = [
            GA(mutation_rate=0.1, tournament_size=3),  # محافظه‌کار
            GA(mutation_rate=0.3, tournament_size=7),  # تهاجمی
            GA(mutation_rate=0.2, tournament_size=5),  # متعادل
        ]
```

</div>

### مقایسه با روش کد:

**پارامترهای ثابت + adaptive mutation:**
- ✅ قابل پیش‌بینی
- ✅ آسان برای debug
- ❌ ممکن است بهینه نباشد برای تمام مراحل

---

## 6. نحوه ایجاد جمعیت اولیه (Initialization Strategy)

### تأثیر: **متوسط** (اولویت 6)

### روش کد:
<div dir="ltr">

```python
def initialize_population(self, var_count):
    # 25% اعداد صحیح
    # 25% اعداد کوچک  
    # 25% کسرهای معمول
    # 25% اعداد اعشاری
```
</div>

### روش‌های جایگزین:

#### 6.1. Sobol Sequence Initialization

<div dir="ltr">

```python
def sobol_initialization(self, var_count):
    # استفاده از توالی شبه-تصادفی Sobol
    from scipy.stats import qmc
    sampler = qmc.Sobol(d=var_count)
    points = sampler.random(self.pop_size)
    
    # تبدیل به محدوده مطلوب
    population = []
    for point in points:
        individual = [self.var_range[0] + p * (self.var_range[1] - self.var_range[0]) 
                     for p in point]
        population.append(individual)
    return population
```

</div>

#### 6.2. Problem-Specific Seeding

<div dir="ltr">

```python
def smart_initialization(self, var_count):
    population = []
    
    # seed با جواب‌های تحلیلی تقریبی
    analytical_seeds = self.get_analytical_approximations()
    for seed in analytical_seeds:
        for _ in range(5):  # 5 variation از هر seed
            variation = self.add_noise(seed)
            population.append(variation)
    
    # بقیه تصادفی
    while len(population) < self.pop_size:
        population.append(self.random_individual(var_count))
    
    return population
```

</div>

---

## 7. معیارهای توقف (Termination Criteria)

### تأثیر: **کم** (اولویت 7)

### روش کد:

- حداکثر نسل
- یافتن جواب دقیق (fitness > 1e10)
- stagnation limit

### جایگزین‌ها:

- Target fitness threshold
- Time limit
- Function evaluations limit
- Convergence criteria

---

## جمع‌بندی و توصیه‌ها

### برای بهبود کد فعلی:

1. **Selection**: آزمایش SUS برای انتخاب یکنواخت‌تر
2. **Crossover**: اضافه کردن BLX-α برای بهبود اکتشاف
3. **Mutation**: پیاده‌سازی Gaussian mutation برای fine-tuning
4. **Population**: آزمایش Island model برای مسائل پیچیده‌تر

### اولویت‌بندی برای بهبود:

1. **بالا**: بهبود Selection strategy
2. **بالا**: تنوع بیشتر در Crossover
3. **متوسط**: Adaptive parameter control
4. **کم**: بهبود initialization برای مسائل خاص

</div>